In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

InputPath="/kaggle/input/skincancer/Skin cancer ISIC The International Skin Imaging Collaboration"

# %%
lx=[]
ly=[]

nPath = InputPath+"/Train/vascular lesion"
for imageName in tqdm(os.listdir(nPath)):
  image = cv.imread(nPath + "/" + imageName)
  image = cv.resize(image,(512,512))
  lx.append(image)
  ly.append(image)

jPath = InputPath+"/Train/squamous cell carcinoma"
for imageName in tqdm(os.listdir(jPath)):
  image = cv.imread(jPath + "/" + imageName)
  image = cv.resize(image,(512,512))
  lx.append(image)
  ly.append(image)

X=np.array(lx)
Y=np.array(ly)

X = X / 255.0
Y = Y / 255.0

x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


print(X.shape)
print(Y.shape)

inputs = keras.Input(shape=(512,512,3))

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
output = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

model = keras.Model(inputs=inputs, outputs=output)

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train,y_train,batch_size=5, epochs=10, verbose=2)
print("Autoencoder")
model.evaluate(x_test,y_test,batch_size=15,verbose=2)

lx=[]
ly=[]

nPath = InputPath+"/Train/vascular lesion"
for imageName in tqdm(os.listdir(nPath)):
  image = cv.imread(nPath + "/" + imageName)
  image = cv.resize(image,(512,512))
  lx.append(image)
  ly.append([1,0])


jPath = InputPath+"/Train/squamous cell carcinoma"
for imageName in tqdm(os.listdir(jPath)):
  image = cv.imread(jPath + "/" + imageName)
  image = cv.resize(image,(512,512))
  lx.append(image)
  ly.append([0,1])

X=np.array(lx)
Y=np.array(ly)

X = X / 255.0

x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


fixed = keras.Model(inputs=inputs, outputs=model.layers[8].output)

x = fixed.output
x = layers.Flatten()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(768, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(384, activation='relu')(x)
out = layers.Dense(2, activation='sigmoid')(x)

final = keras.Model(inputs=fixed.input, outputs=out)
for layer in final.layers[:8]:
  layer.trainable=False

final.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
final.fit(x_train,y_train,batch_size=5, epochs=10, verbose=2)
print("Neural network")
final.evaluate(x_test,y_test,batch_size=15,verbose=2)


y_pred = final.predict(x_test)
conf = np.zeros((2,2))
for i in range(len(y_pred)):
    k=np.argmax(y_pred[i])
    if k==np.argmax(y_test[i]):
        if k==0:
            conf[0][0]+=1
        else:
            conf[1][1]+=1
    else:
        if k==0:
            conf[0][1]+=1
        else:
            conf[1][0]+=1
print(conf)